## <font color='k'> Import Libraries </font>

In [331]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim import SGD
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## <font color='k'> Reading Dataset and Preprocessing </font>

In [332]:
df=pd.read_csv("Bank_Personal_Loan_Modelling.csv")
df.drop(["ID","ZIP Code"],axis=1,inplace=True)
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [333]:
df=pd.get_dummies(df,columns=["Family","Education"],drop_first=True)

In [334]:
cols_to_normalise=['Age','Experience','Income','CCAvg']
df[cols_to_normalise] = StandardScaler().fit_transform(df[cols_to_normalise])
df.head()

,Age,Experience,Income,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Family_2,Family_3,Family_4,Education_2,Education_3
0,-1.774417,-1.666078,-0.538229,-0.193385,0,0,1,0,0,0,0,0,1,0,0
1,-0.029524,-0.096330,-0.864109,-0.250611,0,0,1,0,0,0,0,1,0,0,0
2,-0.552992,-0.445163,-1.363793,-0.536736,0,0,0,0,0,0,0,0,0,0,0
3,-0.901970,-0.968413,0.569765,0.436091,0,0,0,0,0,0,0,0,0,1,0
4,-0.901970,-1.055621,-0.625130,-0.536736,0,0,0,0,0,1,0,0,1,1,0


## <font color='k'> Model, Loss and Optimizer funtions </font>

In [335]:
class Net(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.L = nn.Sequential(nn.Linear(in_features,out_features),nn.Sigmoid())
    def forward(self,x):
        x = self.L(x)
        return x

In [336]:
model = Net(14, 1)
loss_func = nn.BCELoss()
#optimizer = SGD(model.parameters(), lr=0.000001)
optimizer = Adam(model.parameters())

## <font color='k'> DataLoader </font>

In [337]:
X=(df.drop('Personal Loan',axis=1)).values
y=df['Personal Loan'].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
y_train=y_train.reshape(4000,1)
y_test=y_test.reshape(1000,1)

td=TensorDataset((torch.tensor(X_train)).to(torch.float32),(torch.tensor(y_train)).to(torch.float32))
data=DataLoader(td, batch_size=500)

## <font color='k'> Training </font>

In [338]:
def train(model, epoch, data):
    for i in range(epoch):
        for x, y in data:
            pred = model(x)
            loss = loss_func(pred, y)
            loss.backward()

            #Update paramters
            optimizer.step()
            optimizer.zero_grad()
        if i % 10 == 0:
            print(f"Epoch: {i+1}, Error: {loss}")

In [339]:
train(model, 200, data )

Epoch: 1, Error: 20.871431350708008
Epoch: 11, Error: 6.836034297943115
Epoch: 21, Error: 0.4777296185493469
Epoch: 31, Error: 0.4309858977794647
Epoch: 41, Error: 0.3979954421520233
Epoch: 51, Error: 0.3717564046382904
Epoch: 61, Error: 0.35078996419906616
Epoch: 71, Error: 0.33383801579475403
Epoch: 81, Error: 0.31992727518081665
Epoch: 91, Error: 0.3083258867263794
Epoch: 101, Error: 0.29848846793174744
Epoch: 111, Error: 0.29000818729400635
Epoch: 121, Error: 0.2825808823108673
Epoch: 131, Error: 0.2759784460067749
Epoch: 141, Error: 0.2700294554233551
Epoch: 151, Error: 0.2646048665046692
Epoch: 161, Error: 0.2596074342727661
Epoch: 171, Error: 0.254963755607605
Epoch: 181, Error: 0.25061818957328796
Epoch: 191, Error: 0.24652840197086334


## <font color='k'> Testing </font>

In [340]:
temp=TensorDataset((torch.tensor(X_test)).to(torch.float32),(torch.tensor(y_test)).to(torch.float32))
testdata=DataLoader(temp, batch_size=500)

pred=np.array([])
for x,y in testdata:
    currpred=model(x)
    pred=np.append(pred,((currpred>0.5)*1).detach().numpy())
print("Accuracy :",(y_test.reshape(1,1000)==pred).sum()/10,'%')

Accuracy : 88.3 %
